In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
# import numpy as np
# import pandas as pd
import torch

EMBEDDINGS_NAMES = [
    "polblogs",
    "cs_phds",
    "cora",
    "citeseer",
]

# Copied from notebook 2
SIGNATURES = [
    [(-1, 2), (-1, 2)],  # HH
    [(-1, 2), (0, 2)],  # HE
    [(-1, 2), (1, 2)],  # HS
    [(1, 2), (1, 2)],  # SS
    [(1, 2), (0, 2)],  # SE
    [(-1, 4)],  # H
    [(0, 4)],  # E
    [(1, 4)],  # S
]
SIGNATURES_STR = ["HH", "HE", "HS", "SS", "SE", "H", "E", "S"]
n_trials = 10

bad = []
d_avs = {}
for embedding in EMBEDDINGS_NAMES:
    for signature in SIGNATURES_STR:
        for trial in range(n_trials):
            my_data = torch.load(f"../data/graphs/embeddings/{embedding}/{signature}_{trial}.h5")
            # if np.isnan(my_data).any():
            for key in my_data.keys():
                if torch.isnan(my_data[key]).any():
                    bad.append((embedding, signature, trial, key))
                    print(embedding, signature, trial, key)
print(bad)

[]


/tmp/ipykernel_703169/2526613785.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  my_data = torch.load(f"../data/graphs/embeddings/{embedding}/{signature}_{trial}.h5")


In [32]:
# What is the lowest-distortion signature for each embedding?
dists = pd.DataFrame(columns=["embedding", "signature", "trial", "D_avg"])

for embedding in EMBEDDINGS_NAMES:
    D_true, _, _ = embedders.dataloaders.load(embedding)
    D_true = D_true.to("cuda")
    for signature, sigstr in zip(SIGNATURES, SIGNATURES_STR):
        for trial in range(n_trials):
            my_data = torch.load(f"../data/graphs/embeddings/{embedding}/{sigstr}_{trial}.h5", map_location="cuda", weights_only=True)

            # Get train and test index
            test_idx = my_data["test_idx"]
            train_idx = torch.tensor([i for i in range(D_true.shape[0]) if i not in test_idx], device="cuda")

            # Dist needs to be rescaled
            D_train = D_true[train_idx][:, train_idx]
            D_train_max = torch.max(D_train[torch.isfinite(D_train)])
            D_train = D_train / D_train_max

            # Get estimated distance
            pm = embedders.manifolds.ProductManifold(signature=signature, device="cuda")
            D_est = pm.pdist(my_data["X_train"])

            # Correct for scaling using maximum-likelihood scaling factor
            triu_idx = torch.triu_indices(D_train.shape[0], D_train.shape[1], offset=1)
            D_train_triu = D_train[triu_idx[0], triu_idx[1]]
            D_est_triu = D_est[triu_idx[0], triu_idx[1]]
            a = (
                torch.sum((D_train_triu * D_est_triu) / (D_train_triu ** 2)) / 
                torch.sum(D_est_triu ** 2 / D_train_triu ** 2)
            )
            D_est = a * D_est

            D_avg = embedders.metrics.d_avg(D_train, D_est, pairwise=True).item()
            print(D_avg)

            dists.loc[len(dists)] = {
                    "embedding": embedding,
                    "signature": signature,
                    "trial": trial,
                    "D_avg": D_avg
                }

Top CC has 1222 nodes; original graph has 1490 nodes.
8133.362760749074
6282.458146391739
6514.013792551637
2817.579299868043
5138.478426278824
7488.2461157228
10689.239262650774
6901.124861788506
7829.991511053375
5125.661594368471
11.115766913300245
9.417566351510077
20.26462634401046
10.764127569406167
17.33344820301262
6.631368254330658
11.73347137557037
13.085465401050538
13.777944691529898
9.18694457271293
1.5801888580075505
1.5447004709422136
1.4578341312042675
1.5926130733556012
1.7024469998140357
1.599671478881776
1.668014194045843
1.723744060004334
1.6548367323561433
1.9222265751346213
1.2977793270399474
1.2041149751143094
1.451637327589419
1.2800961342302954
1.3325622458136908
1.3959049222198978
1.4146348229193468
1.3254927531838168
1.458189406944395
1.2937367773994906
1.7414195720374466
1.836662726702543
1.6952059555901462
1.547037583373384
1.9439169488232702
1.6143182558341176
1.4742973853744141
1.658631345553393
1.6878575447991249
1.669320195591129
16086.808469667158
2532

In [31]:
a

tensor(nan, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

In [18]:
D_train

tensor([[0.0000, 0.1250, 0.3750,  ..., 0.2500, 0.5000, 0.3750],
        [0.1250, 0.0000, 0.3750,  ..., 0.3750, 0.5000, 0.3750],
        [0.3750, 0.3750, 0.0000,  ..., 0.2500, 0.3750, 0.2500],
        ...,
        [0.2500, 0.3750, 0.2500,  ..., 0.0000, 0.3750, 0.2500],
        [0.5000, 0.5000, 0.3750,  ..., 0.3750, 0.0000, 0.1250],
        [0.3750, 0.3750, 0.2500,  ..., 0.2500, 0.1250, 0.0000]],
       device='cuda:0', dtype=torch.float64)

In [22]:
D_train.isfinite().all()

tensor(True, device='cuda:0')

In [20]:
embedders.metrics.d_avg(D_train, D_est)

tensor(nan, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)

In [25]:
torch.abs(D_est - D_train) / D_train

tensor([[   nan, 1.9749, 0.3306,  ..., 0.5305, 0.4927, 0.2669],
        [1.9749,    nan, 0.4439,  ..., 0.0091, 0.0212, 0.2882],
        [0.3306, 0.4439,    nan,  ..., 0.5147, 0.1386, 0.6107],
        ...,
        [0.5305, 0.0091, 0.5147,  ...,    nan, 0.0066, 0.0089],
        [0.4927, 0.0212, 0.1386,  ..., 0.0066,    nan, 1.5516],
        [0.2669, 0.2882, 0.6107,  ..., 0.0089, 1.5516,    nan]],
       device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

In [39]:
import embedders
import pandas as pd
from tqdm.notebook import tqdm

# MAX_DEPTH = 3
# MAX_DEPTH = None
# N_FEATURES = "d_choose_2"

CLS_NAMES = ["polblogs", "cora", "citeseer"]

SIGNATURES = [
    [(1, 2), (1, 2)], # SS
    [(-1, 4)], # H
    [(-1, 2), (1, 2)], # HS
]

SIGNATURES_STR = ["SS", "H", "HS"]

results = []
my_tqdm = tqdm(total=len(CLS_NAMES) * n_trials)# * len(SIGNATURES))
for embedding, signature, sigstr in zip(CLS_NAMES, SIGNATURES, SIGNATURES_STR):
    _, _, adj = embedders.dataloaders.load(embedding)
    # dists = pd.read_table(f"../data/graphs/embeddings/{embedding}_dists.tsv")
    
    # Get the lowest D_avg signature
    
    # for sig, sig_str in zip(SIGNATURES, SIGNATURES_STR):
    pm = embedders.manifolds.ProductManifold(signature=sig)
    # dists_sig = dists[dists["signature"] == str(sig)]

    # for i, trial in enumerate(dists_sig["seed"]):
    for i in range(n_trials):
        # i is used to load the correct embedding
        # trial is used to get the correct d_avg
        # I'm sorry for this mess
        data = torch.load(f"../data/graphs/embeddings/{embedding}/{sig_str}_{i}.h5", weights_only=True)
        X_train = data["X_train"]
        X_test = data["X_test"]
        y_train = data["y_train"]
        y_test = data["y_test"]
        test_idx = data["test_idx"]
        train_idx = [i for i in range(len(X_train) + len(X_test)) if i not in test_idx]

        # Get adjacency matrix
        A_hat = embedders.predictors.kappa_gcn.get_A_hat(adj.float())

        res = embedders.benchmarks.benchmark(
            X=None,
            y=None,
            X_train=X_train,
            X_test=X_test,
            y_train=y_train,
            y_test=y_test,
            pm=pm,
            device="cuda",
            A_train = A_hat[train_idx][:, train_idx],
            A_test = A_hat[test_idx][:, test_idx],
            task="classification",
            score=["accuracy", "f1-micro"],
        )
        res["embedding"] = embedding
        res["signature"] = sig_str
        res["trial"] = trial

        # Get matching d_avg using seed and sig_str
        # dists_seed = dists_sig[dists_sig["seed"] == trial]
        # assert len(dists_seed) == 1
        # res["d_avg"] = dists_seed["d_avg"].values[0]

        results.append(res)
        my_tqdm.update(1)

results = pd.DataFrame(results)

  0%|          | 0/30 [00:00<?, ?it/s]

Top CC has 1222 nodes; original graph has 1490 nodes.


  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

...

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

In [40]:
results.to_csv("../data/results_icml/graph_classification.tsv", sep="\t", index=False)

In [42]:
REG_NAMES = ["cs_phds"]

SIGNATURES = [
    [(-1, 4)], # H
]

SIGNATURES_STR = ["H"]

results2 = []
my_tqdm = tqdm(total=len(REG_NAMES) * n_trials)# * len(SIGNATURES))
for embedding, signature, sigstr in zip(REG_NAMES, SIGNATURES, SIGNATURES_STR):
    _, _, adj = embedders.dataloaders.load(embedding)
    # dists = pd.read_table(f"../data/graphs/embeddings/{embedding}_dists.tsv")
    
    # Get the lowest D_avg signature
    
    # for sig, sig_str in zip(SIGNATURES, SIGNATURES_STR):
    pm = embedders.manifolds.ProductManifold(signature=sig)
    # dists_sig = dists[dists["signature"] == str(sig)]

    # for i, trial in enumerate(dists_sig["seed"]):
    for i in range(n_trials):
        # i is used to load the correct embedding
        # trial is used to get the correct d_avg
        # I'm sorry for this mess
        data = torch.load(f"../data/graphs/embeddings/{embedding}/{sig_str}_{i}.h5", weights_only=True)
        X_train = data["X_train"]
        X_test = data["X_test"]
        y_train = data["y_train"]
        y_test = data["y_test"]
        test_idx = data["test_idx"]
        train_idx = [i for i in range(len(X_train) + len(X_test)) if i not in test_idx]

        # Get adjacency matrix
        A_hat = embedders.predictors.kappa_gcn.get_A_hat(adj.float())

        res = embedders.benchmarks.benchmark(
            X=None,
            y=None,
            X_train=X_train,
            X_test=X_test,
            y_train=y_train,
            y_test=y_test,
            pm=pm,
            device="cuda",
            A_train = A_hat[train_idx][:, train_idx],
            A_test = A_hat[test_idx][:, test_idx],
            # task="classification",
            # score=["accuracy", "f1-micro"],
            task="regression",
            score=["rmse"]
        )
        res["embedding"] = embedding
        res["signature"] = sig_str
        res["trial"] = trial

        # Get matching d_avg using seed and sig_str
        # dists_seed = dists_sig[dists_sig["seed"] == trial]
        # assert len(dists_seed) == 1
        # res["d_avg"] = dists_seed["d_avg"].values[0]

        results2.append(res)
        my_tqdm.update(1)

results2 = pd.DataFrame(results2)

  0%|          | 0/10 [00:00<?, ?it/s]

Top CC has 1025 nodes; original graph has 1025 nodes.


  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

...

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

In [43]:
results2.to_csv("../data/results_icml/graph_regression.tsv", sep="\t", index=False)

In [23]:
results.groupby(["embedding", "signature"]).mean()

sklearn_dt  sklearn_rf  product_dt  product_rf  \
embedding signature                                                   
citeseer  E            0.224645    0.246919    0.231043    0.251185   
          H            0.224171    0.236019    0.216351    0.249526   
          HE           0.224408    0.244076    0.232464    0.236019   
          HH           0.223223    0.241469    0.221564    0.237678   
          HS           0.228199    0.247867    0.227014    0.242891   
          S            0.227962    0.231517    0.235308    0.235071   
          SE           0.226777    0.229621    0.222275    0.234834   
          SS           0.221090    0.244787    0.230095    0.238152   
cora      E            0.193763    0.208048    0.196579    0.209859   
          H            0.188732    0.199396    0.182495    0.200805   
          HE           0.192555    0.201207    0.204024    0.214487   
          HH           0.196781    0.212072    0.191952    0.213883   
          HS           0.191348    0.218511    0.194769    0.215292   
          S            0.190342    0.198994    0.189336    0.202616   
          SE           0.194366    0.213280    0.184306    0.216097   
          SS           0.184909    0.206237    0.178270    0.209859   
cs_phds   E            0.071220    0.063902    0.067805    0.070732   
          H            0.076585    0.076585    0.072195    0.075610   
          HE           0.077561    0.080000    0.063902    0.073659   
          HH           0.074797    0.083469    0.073713    0.075339   
          HS           0.060000    0.066341    0.060000    0.065854   
          S            0.061951    0.063415    0.060976    0.071220   
          SE           0.061951    0.069268    0.062439    0.067317   
          SS           0.058049    0.076098    0.062439    0.068780   
polblogs  E            0.905714    0.931020    0.909796    0.936735   
          H            0.914286    0.937143    0.908163    0.935918   
          HE           0.903265    0.929796    0.913878    0.933878   
          HH           0.899592    0.929388    0.908980    0.924490   
          HS           0.911837    0.933061    0.904490    0.935102   
          S            0.917551    0.933878    0.915510    0.928163   
          SE           0.915102    0.930612    0.904082    0.929388   
          SS           0.908571    0.937143    0.922041    0.935102   

                     tangent_dt  tangent_rf       knn  ps_perceptron  trial  \
embedding signature                                                           
citeseer  E            0.221801    0.240047  0.246445       0.136256    4.5   
          H            0.222986    0.240284  0.251896       0.138626    4.5   
          HE           0.225829    0.246682  0.240995       0.139573    4.5   
          HH           0.214455    0.245972  0.254502       0.055213    4.5   
          HS           0.224408    0.243365  0.260900       0.132227    4.5   
          S            0.230569    0.240047  0.243365       0.138152   18.5   
          SE           0.218246    0.237915  0.246682       0.132701    4.5   
          SS           0.231280    0.235545  0.253318       0.139336    4.5   
cora      E            0.188330    0.207243  0.206439       0.172837   64.5   
          H            0.176056    0.198994  0.207042       0.181087   54.5   
          HE           0.195976    0.208249  0.218712       0.175855   14.5   
          HH           0.187726    0.207847  0.204628       0.051509    4.5   
          HS           0.190141    0.208451  0.220322       0.163380   24.5   
          S            0.189336    0.210463  0.212877       0.164588   74.5   
          SE           0.197586    0.207243  0.207445       0.184708   44.5   
          SS           0.188330    0.210060  0.207646       0.201408   34.5   
cs_phds   E            0.064878    0.064390  0.057561       0.039512   64.5   
          H            0.067805    0.079512  0.065366       0.049268   54.5   
          HE           0.0